In [1]:
from dotenv import load_dotenv
load_dotenv()



True

In [11]:
# ================================================
# CREWAI TEST CASE GENERATOR (SAVES TO DOWNLOADS)
# ================================================

from crewai import LLM, Agent, Crew, Task
import pandas as pd
import re
import os
from io import StringIO

# --- Create LLM instance ---
llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.1)

# --- Agent 1: Requirement Analyzer ---
requirement_agent = Agent(
    name="RequirementAgent",
    role="Requirement Analyzer",
    goal="Analyze the acceptance criteria and extract all testable requirements.",
    backstory="You are an experienced QA Analyst skilled at identifying testable conditions and edge cases.",
    llm=llm
)

# --- Agent 2: Test Case Generator ---
testcase_agent = Agent(
    name="TestCaseAgent",
    role="Test Case Generator",
    goal="Generate structured test cases including name, precondition, description, expected result, and steps.",
    backstory="You are a QA automation expert who creates reusable and well-structured test cases based on requirements.",
    llm=llm
)

# --- Agent 3: Reviewer & Formatter ---
reviewer_agent = Agent(
    name="ReviewerAgent",
    role="Reviewer and Formatter",
    goal="Review the test cases for clarity, grammar, and correct markdown formatting.",
    backstory="You are a senior QA reviewer who ensures test documentation quality and consistency.",
    llm=llm
)

# --- Task 1: Extract Requirements ---
task1 = Task(
    description=(
        "Extract all possible testable requirements from the following acceptance criteria:\n\n"
        "{acceptance_criteria}\n\n"
        "Focus on functional and edge cases."
    ),
    agent=requirement_agent,
    expected_output="A list of clear, testable requirements."
)

# --- Task 2: Generate Test Cases ---
task2 = Task(
    description=(
        "Based on the extracted requirements below:\n\n"
        "{task1.output}\n\n"
        "Write test cases in this exact format:\n"
        "| Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |\n"
    ),
    agent=testcase_agent,
    expected_output="Test cases written in a markdown table format."
)

# --- Task 3: Review and Format ---
task3 = Task(
    description=(
        "Review and clean up the generated test cases for clarity, formatting, and structure:\n\n"
        "{task2.output}"
    ),
    agent=reviewer_agent,
    expected_output="Final formatted markdown table of validated test cases."
)

# --- Create Crew ---
crew = Crew(
    agents=[requirement_agent, testcase_agent, reviewer_agent],
    tasks=[task1, task2, task3],
    process="sequential",
    verbose=True
)

# --- Input Acceptance Criteria ---
acceptance_criteria = """
When the user logs in with valid credentials, they should be redirected to the dashboard.
If credentials are invalid, an error message should be displayed.
User sessions should persist until logout or session timeout.
"""

# --- Run the Crew ---
result = crew.kickoff(inputs={"acceptance_criteria": acceptance_criteria})

# --- Handle CrewAI version differences safely ---
final_output = (
    getattr(result, "final_output", None)
    or getattr(result, "output", None)
    or getattr(result, "raw", None)
    or str(result)
)

print("\n✅ Final Generated Test Cases (Markdown):\n")
print(final_output)

# --- Extract Markdown Table ---
table_match = re.search(r"(\| *Test Case Name.*)", final_output, re.DOTALL)

if table_match:
    table_text = table_match.group(1).strip()
    lines = [line.strip() for line in table_text.splitlines() if line.strip().startswith("|")]

    # --- Split into rows and columns ---
    split_rows = [[col.strip() for col in line.split("|") if col.strip()] for line in lines]
    split_rows = [row for row in split_rows if not all(re.match(r"-+", cell) for cell in row)]

    max_cols = max(len(r) for r in split_rows)
    for r in split_rows:
        while len(r) < max_cols:
            r.append("")

    header = split_rows[0]
    rows = split_rows[1:]
    df = pd.DataFrame(rows, columns=header)

    # --- Add auto numbering ---
    df.insert(0, "TC_ID", [f"TC{str(i+1).zfill(3)}" for i in range(len(df))])

    # --- Save to Downloads folder ---
    downloads_dir = os.path.join(os.path.expanduser("~"), "Downloads")
    os.makedirs(downloads_dir, exist_ok=True)
    output_path = os.path.join(downloads_dir, "generated_test_cases.xlsx")

    df.to_excel(output_path, index=False)
    print(f"\n✅ Excel file saved successfully in your Downloads folder:\n{output_path}")

else:
    print("\n⚠️ No markdown table found in the output. Please check Crew output formatting.")


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 68810482-6966-456b-9b5a-0ca1f5ee64d3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/satishmokkala/Testcase-Creation/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Requirement Analyzer                                                                                    │
│                                                                                                                 │
│  Task: Extract all possible testable requirements from the following acceptance criteria:                       │
│                                                                                                                 │
│                                                                                                                 │
│  When the user logs in with valid credentials, they should be redirected to the dashboard.                      │
│  If credentials are invalid, an error message should be displayed.                                              │
│  User sessions should persist until logout or session timeout.                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Focus on functional and edge cases.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cc162398-e157-4ebb-ad60-d321a00d1b7d                                                                     │
│  Agent: Requirement Analyzer                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Requirement Analyzer                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Functional Requirements:**                                                                                   │
│                                                                                                                 │
│  1.  **Successful Login:**                                                                                      │
│      *   **Requirement:** When a user enters valid credentials (username and password), the system must         │
│  authenticate the user.                                                                                         │
│      *   **Testable Condition:** Input valid username and password. Verify successful authentication.           │
│      *   **Expected Result:** User is authenticated and redirected to the dashboard.                            │
│                                                                                                                 │
│  2.  **Dashboard Redirection:**                                                                                 │
│      *   **Requirement:** Upon successful login, the user must be redirected to the dashboard.                  │
│      *   **Testable Condition:** After successful login, verify the page URL or title.                          │
│      *   **Expected Result:** The user is redirected to the dashboard page.                                     │
│                                                                                                                 │
│  3.  **Invalid Credentials - Error Message Display:**                                                           │
│      *   **Requirement:** When a user enters invalid credentials, an error message must be displayed.           │
│      *   **Testable Condition:** Input invalid username and/or password.                                        │
│      *   **Expected Result:** An error message is displayed on the login page indicating invalid credentials.   │
│                                                                                                                 │
│  4.  **Error Message Content:**                                                                                 │
│      *   **Requirement:** The error message displayed for invalid credentials must be clear and informative.    │
│      *   **Testable Condition:** Input invalid credentials. Inspect the error message text.                     │
│      *   **Expected Result:** The error message clearly states that the username or password is incorrect.      │
│                                                                                                                 │
│  5.  **Session Persistence:**                                                                                   │
│      *   **Requirement:** User sessions should persist after successful login.                                  │
│      *   **Testable Condition:** Log in successfully. Navigate to different pages within the application.       │
│  Close and reopen the browser.                                                                                  │
│      *   **Expected Result:** The user remains logged in and can access the application without re-entering     │
│  credentials.                                                                                                   │
│                                                        

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Reviewer and Formatter                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  | Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |                      │
│  |---|---|---|---|                                                                                              │
│  | TC_Login_001_SuccessfulLogin | User account exists with valid credentials. | User is authenticated and       │
│  redirected to the dashboard upon entering valid credentials. | 1. Open the login page. 2. Enter a valid        │
│  username. 3. Enter a valid password. 4. Click the "Login" button. 5. Verify redirection to the dashboard. |    │
│  | TC_Login_002_DashboardRedirection | User is successfully logged in. | User is redirected to the dashboard    │
│  page after successful login. | 1. Perform steps in TC_Login_001. 2. Verify the URL or title of the page        │
│  matches the dashboard page. |                                                                                  │
│  | TC_Login_003_InvalidCredentials_ErrorMessage | None | An error message is displayed when the user enters     │
│  invalid credentials. | 1. Open the login page. 2. Enter an invalid username and/or password. 3. Click the      │
│  "Login" button. 4. Verify that an error message is displayed on the login page. |                              │
│  | TC_Login_004_ErrorMessageContent | None | The error message clearly states that the username or password is  │
│  incorrect. | 1. Perform steps in TC_Login_003. 2. Inspect the text of the error message. 3. Verify that the    │
│  error message indicates invalid username or password. |                                                        │
│  | TC_Login_005_SessionPersistence | User is successfully logged in. | User session persists after successful   │
│  login, even after navigating to different pages and closing/reopening the browser. | 1. Perform steps in       │
│  TC_Login_001. 2. Navigate to different pages within the application. 3. Close the browser. 4. Reopen the       │
│  browser and navigate to the application. 5. Verify that the user remains logged in and can access the          │
│  application. |                                                                                                 │
│  | TC_Login_006_LogoutFunctionality | User is successfully logged in. | User is logged out, the session is      │
│  terminated, and the user is redirected to the login page after clicking the logout button. | 1. Perform steps  │
│  in TC_Login_001. 2. Click the "Logout" button/link. 3. Verify that the user is redirected to the login page    │
│  or a public page. |                                                                                            │
│  | TC_Login_007_SessionTimeout | User is successfully logged in. | User is automatically logged out after a     │
│  defined period of inactivity (session timeout). | 1. Perform steps in TC_Login_001. 2. Remain inactive for     │
│  the duration of the session timeout period. 3. Verify that the user is automatically logged out and            │
│  redirected to the login page or a session timeout page. |                                                      │
│  | TC_Login_008_InvalidCredentials_UsernameOnly | None | An appropriate error message is displayed when only    │
│  the username is invalid. | 1. Open the login page. 2. 


✅ Final Generated Test Cases (Markdown):

```
| Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |
|---|---|---|---|
| TC_Login_001_SuccessfulLogin | User account exists with valid credentials. | User is authenticated and redirected to the dashboard upon entering valid credentials. | 1. Open the login page. 2. Enter a valid username. 3. Enter a valid password. 4. Click the "Login" button. 5. Verify redirection to the dashboard. |
| TC_Login_002_DashboardRedirection | User is successfully logged in. | User is redirected to the dashboard page after successful login. | 1. Perform steps in TC_Login_001. 2. Verify the URL or title of the page matches the dashboard page. |
| TC_Login_003_InvalidCredentials_ErrorMessage | None | An error message is displayed when the user enters invalid credentials. | 1. Open the login page. 2. Enter an invalid username and/or password. 3. Click the "Login" button. 4. Verify that an error message is displayed on the login

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 17f03075-fe9e-4bde-8e5a-5907d246aa67                                                                     │
│  Agent: Reviewer and Formatter                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 68810482-6966-456b-9b5a-0ca1f5ee64d3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  | Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |                      │
│  |---|---|---|---|                                                                                              │
│  | TC_Login_001_SuccessfulLogin | User account exists with valid credentials. | User is authenticated and       │
│  redirected to the dashboard upon entering valid credentials. | 1. Open the login page. 2. Enter a valid        │
│  username. 3. Enter a valid password. 4. Click the "Login" button. 5. Verify redirection to the dashboard. |    │
│  | TC_Login_002_DashboardRedirection | User is successfully logged in. | User is redirected to the dashboard    │
│  page after successful login. | 1. Perform steps in TC_Login_001. 2. Verify the URL or title of the page        │
│  matches the dashboard page. |                                                                                  │
│  | TC_Login_003_InvalidCredentials_ErrorMessage | None | An error message is displayed when the user enters     │
│  invalid credentials. | 1. Open the login page. 2. Enter an invalid username and/or password. 3. Click the      │
│  "Login" button. 4. Verify that an error message is displayed on the login page. |                              │
│  | TC_Login_004_ErrorMessageContent | None | The error message clearly states that the username or password is  │
│  incorrect. | 1. Perform steps in TC_Login_003. 2. Inspect the text of the error message. 3. Verify that the    │
│  error message indicates invalid username or password. |                                                        │
│  | TC_Login_005_SessionPersistence | User is successfully logged in. | User session persists after successful   │
│  login, even after navigating to different pages and closing/reopening the browser. | 1. Perform steps in       │
│  TC_Login_001. 2. Navigate to different pages within the application. 3. Close the browser. 4. Reopen the       │
│  browser and navigate to the application. 5. Verify that the user remains logged in and can access the          │
│  application. |                                                                                                 │
│  | TC_Login_006_LogoutFunctionality | User is successfully logged in. | User is logged out, the session is      │
│  terminated, and the user is redirected to the login page after clicking the logout button. | 1. Perform steps  │
│  in TC_Login_001. 2. Click the "Logout" button/link. 3. Verify that the user is redirected to the login page    │
│  or a public page. |                                                                                            │
│  | TC_Login_007_SessionTimeout | User is successfully logged in. | User is automatically logged out after a     │
│  defined period of inactivity (session timeout). | 1. Perform steps in TC_Login_001. 2. Remain inactive for     │
│  the duration of the session timeout period. 3. Verify that the user is automatically logged out and            │
│  redirected to the login page or a session timeout page. |                                                      │
│  | TC_Login_008_InvalidCredentials_UsernameOnly | None


✅ Excel file saved successfully in your Downloads folder:
/Users/satishmokkala/Downloads/generated_test_cases.xlsx
